In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from matplotlib import pyplot as plt
import numpy as np
# from pyspark.sql.functions import avg, stddev_pop
import pyspark.sql.functions as sparkfun

In [ ]:
text_path = r"./itcont_2016_20161204_92060702.txt"
heaher_path = r"./indiv_header_file.csv"


In [ ]:
reader = spark.read 
reader.option("header",True) 
reader.option("inferSchema",True) 
reader.option("sep", "|") 
# reader.option("delimter", "|") 
donation_df = reader.csv(text_path)

reader = spark.read 
reader.option("header",True) 
reader.option("inferSchema",True) 
reader.option("delimter", "|") 
header_df = reader.csv(heaher_path)

donation_df =  donation_df.toDF(*header_df.columns)

In [ ]:
donation_df = donation_df.select("CMTE_ID","TRANSACTION_TP","ENTITY_TP","TRANSACTION_AMT")
donation_df.show()

In [ ]:
donation_transaction_list =["15","15E","17R","18K","18L","18U","19"]

# Question 1
# 1. How many donations did each campaign have?

In [195]:
# three_cand_count = donation_df[donation_df.TRANSACTION_TP.isin( donation_transaction_list)] # temporary
three_cand_count = donation_df
three_cand_count = three_cand_count.groupBy('CMTE_ID')\
    .agg( sparkfun.count('CMTE_ID'))

result_1 = three_cand_count[three_cand_count.CMTE_ID.isin(["C00575795","C00577130","C00580100"])]
result_1.show()

+---------+--------------+
|  CMTE_ID|count(CMTE_ID)|
+---------+--------------+
|C00577130|            10|
|C00575795|           637|
|C00580100|          1417|
+---------+--------------+



# 2. What was the total amount donated to each campaign?

In [ ]:
three_cand_amt = donation_df.groupBy('CMTE_ID')\
    .agg( sparkfun.sum('TRANSACTION_AMT') )
# result.show()

result_2 = three_cand_amt[three_cand_amt.CMTE_ID.isin(["C00575795","C00577130","C00580100"])]
result_2.show()

In [ ]:
new_df = result_1.join(result_2, on=['CMTE_ID'])
new_df.show()

# 3. What percentage of the each campaign’s donations was done by small contributors?

In [238]:
ind_df = donation_df.groupBy("CMTE_ID", "ENTITY_TP")\
    .agg( sparkfun.count('CMTE_ID') )\
    .where((sparkfun.col("CMTE_ID") == "C00575795") | (sparkfun.col("CMTE_ID") == "C00577130") | \
           (sparkfun.col("CMTE_ID") == "C00580100") & (sparkfun.col("ENTITY_TP") == "IND"))
total_entity_df = donation_df.groupBy("CMTE_ID")\
    .agg( sparkfun.count('ENTITY_TP') )\
    .where((sparkfun.col("CMTE_ID") == "C00575795") | (sparkfun.col("CMTE_ID") == "C00577130") | \
           (sparkfun.col("CMTE_ID") == "C00580100"))

new_df =  ind_df.join(total_entity_df, on=["CMTE_ID"])


new_df = new_df.withColumn('percentage',new_df["count(CMTE_ID)"]/new_df["count(ENTITY_TP)"] )
new_df.show()

+---------+---------+--------------+----------------+------------------+
|  CMTE_ID|ENTITY_TP|count(CMTE_ID)|count(ENTITY_TP)|        percentage|
+---------+---------+--------------+----------------+------------------+
|C00580100|      IND|          1414|            1415|0.9992932862190813|
+---------+---------+--------------+----------------+------------------+



# 4. Produce a histogram of the donations for each campaign?

# Code for AWS

In [204]:
def files_from_args(): 
	import argparse
	parser = argparse.ArgumentParser() 
	parser.add_argument('-f', '--file', default='file')
	parser.add_argument('-c', '--column', default='column') 
	parser.add_argument('-o', '--output',default='output') 
	# parser.add_argument('-i', '--input', default='input') 

	args = parser.parse_args() 
	return (args.file, args.column, args.output) 

In [226]:
def gradandclearthedat(file_path,heaher_path):
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()
    
    reader = spark.read 
    reader.option("header",True) 
    reader.option("inferSchema",True) 
    reader.option("sep", "|") 
    # reader.option("delimter", "|") 
    donation_df = reader.csv(file_path)

    reader = spark.read 
    reader.option("header",True) 
    reader.option("inferSchema",True) 
    reader.option("delimter", "|") 
    header_df = reader.csv(heaher_path)
    donation_df =  donation_df.toDF(*header_df.columns)
    donation_df = donation_df.select("CMTE_ID","TRANSACTION_TP","ENTITY_TP","TRANSACTION_AMT")
    return donation_df

In [255]:
def count_donation(donation_df,output):
    import pyspark.sql.functions as sparkfun
    donation_transaction_list =["15","15E","17R","18K","18L","18U","19"]    
    donation_df = donation_df[donation_df.TRANSACTION_TP.isin( donation_transaction_list)]
    three_cand_count = donation_df.groupBy('CMTE_ID')        .agg( sparkfun.count('CMTE_ID'))
    result_1 = three_cand_count[three_cand_count.CMTE_ID.isin(["C00575795","C00577130","C00580100"])]
    print("The count of donations for each campaign")
    result_1.show()
    result_1 = result_1.repartition(1)
    count_output = output + "-count"
    result_1.write.format('csv').save(count_output)   
    return donation_df

In [254]:
def amount_donation(donation_df,output):
    import pyspark.sql.functions as sparkfun
    three_cand_amt = donation_df.groupBy('CMTE_ID')\
    .agg( sparkfun.sum('TRANSACTION_AMT') )
    result_2 = three_cand_amt[three_cand_amt.CMTE_ID.isin(["C00575795","C00577130","C00580100"])]
    
    print("The amount of donations for each campaign")
    result_2.show()
    result_2 = result_2.repartition(1)
    amount_output = output + "-amount"
    result_2.write.format('csv').save(amount_output)

In [253]:
def percentage_donation(donation_df,output):
    import pyspark.sql.functions as sparkfun
    ind_df = donation_df.groupBy("CMTE_ID", "ENTITY_TP")\
        .agg( sparkfun.count('CMTE_ID') )\
        .where((sparkfun.col("CMTE_ID") == "C00575795") | (sparkfun.col("CMTE_ID") == "C00577130") | \
               (sparkfun.col("CMTE_ID") == "C00580100") & (sparkfun.col("ENTITY_TP") == "IND"))
    total_entity_df = donation_df.groupBy("CMTE_ID")\
        .agg( sparkfun.count('ENTITY_TP') )\
        .where((sparkfun.col("CMTE_ID") == "C00575795") | (sparkfun.col("CMTE_ID") == "C00577130") | \
               (sparkfun.col("CMTE_ID") == "C00580100"))
    result_3 =  ind_df.join(total_entity_df, on=["CMTE_ID"])
    result_3 = result_3.withColumn('percentage',result_3["count(CMTE_ID)"]/result_3["count(ENTITY_TP)"] )
    
    print("The percentage of small contributors each campaign")
    result_3.show()
    result_3 = result_3.repartition(1)
    percent_output = output + "-percent"
    result_3.write.format('csv').save(percent_output)    

In [ ]:
if __name__ == "__main__": 
    file_path, heaher_path,output = files_from_args() 
    donation_df = gradandclearthedat(file_path,heaher_path)
    donation_df = count_donation(donation_df,output)  
    amount_donation(donation_df,output)
    percentage_donation(donation_df,output)    

In [252]:
## Test code

donation_df = gradandclearthedat(text_path,heaher_path)
donation_df = count_donation(donation_df,"")  
amount_donation(donation_df,"")  

percentage_donation(donation_df,"")

The count of donations for each campaign
+---------+--------------+
|  CMTE_ID|count(CMTE_ID)|
+---------+--------------+
|C00580100|          1415|
+---------+--------------+

The amount of donations for each campaign
+---------+--------------------+
|  CMTE_ID|sum(TRANSACTION_AMT)|
+---------+--------------------+
|C00580100|              201384|
+---------+--------------------+

The percentage of small contributors each campaign
+---------+---------+--------------+----------------+------------------+
|  CMTE_ID|ENTITY_TP|count(CMTE_ID)|count(ENTITY_TP)|        percentage|
+---------+---------+--------------+----------------+------------------+
|C00580100|      IND|          1414|            1415|0.9992932862190813|
+---------+---------+--------------+----------------+------------------+

